## API 구분


- [네이버](https://developers.naver.com/docs/common/openapiguide/)
    - 통합 검색어 트렌드 API
    - 쇼핑인사이트 API

- [클라우드 API](https://api.ncloud-docs.com/docs/api-overview)
    - Geocoding
    - Reverse Geocoding
    - Static Map

In [ ]:
from config import config

# 네이버 파파고 API
naver_id = config['naver']['client_id']
naver_pw = config['naver']['client_secret']

# 네이버 클라우드 플랫폼 API
cloud_id = config['cloud']['client_id']
cloud_pw = config['cloud']['client_secret']

In [ ]:
import json
import datetime
import requests
import pandas as pd

## MAP

In [ ]:
class Map:
    """
    네이버 지도 API 클래스
    """
    def _init_(self):
        pass

    def transit_directions_point_to_point(self, **kwargs):
        """
        네이버 지도 길찾기 대중교통 API
        """
        url = "https://map.naver.com/v5/api/transit/directions/point-to-point?"
        params = {
            "crs": "EPSG:4326",
            "mode": "TIME",
            "lang": "ko",
            "includeDetailOperation": "true",
        }
        kwargs.update(params)
        for k, v in kwargs.items():
            url += f"{k}={v}&"
        res = requests.get(url)
        if res.status_code == 200:
            return res.json()
        else:
            return res

### 지도 길찾기 대중교통

In [ ]:
# 네이버 지도 API 인스턴스
maps = Map()

# 파라미터
sx, sy = 127.019534, 37.51608
ex, ey = 127.111203, 37.39475
start = f"{sx},{sy}"
goal = f"{ex},{ey}"
today = datetime.date.today()
departureTime = (today + datetime.timedelta(days=-today.weekday(), weeks=1)).strftime("%Y-%m-%dT08:00:00")

# API 호출
data = maps.transit_directions_point_to_point(start=start,
                                         goal=goal,
                                         departureTime=departureTime)
data

## NAVER

In [ ]:
class Naver:
    """
    네이버 OPEN API 클래스
    """

    def __init__(self, client_id, client_secret):
        """
        API 인증 정보 초기화
        """
        self.headers = {
            "X-Naver-Client-Id": client_id,
            "X-Naver-Client-Secret": client_secret,
            "Content-Type": "application/json",
        }

    def datalab_search(self, **kwargs):
        """
        네이버 통합 검색어 트렌드 조회
        - API 레퍼런스: https://developers.naver.com/docs/serviceapi/datalab/search/search.md
        """
        url = "https://openapi.naver.com/v1/datalab/search"
        data = {}
        for k, v in kwargs.items():
            data[k] = v
        data = json.dumps(data, ensure_ascii=False).encode("utf-8")
        res = requests.post(url, data=data, headers=self.headers)

        if res.status_code == 200:
            data = res.json()['results']
            num = len(data)
            df = pd.DataFrame()
            for i in range(num):
                sub = pd.DataFrame(data[i]['data'])
                sub['title'] = data[i]['title']
                sub = sub[['title', 'period', 'ratio']]
                df = pd.concat([df, sub], axis=0, ignore_index=True)
            pivot = pd.pivot(df, index='period', columns='title',
                             values='ratio').reset_index()
            pivot.columns.name = None
            pivot = pivot.rename(columns={"period": "날짜"})
            return pivot

        else:
            return res

    def datalab_shopping_categories(self, **kwargs):
        """
        쇼핑인사이트 분야별 트렌드 조회
        - API 레퍼런스: https://developers.naver.com/docs/serviceapi/datalab/shopping/shopping.md
        """
        url = "https://openapi.naver.com/v1/datalab/shopping/categories"
        data = {}
        for k, v in kwargs.items():
            data[k] = v
        data = json.dumps(data, ensure_ascii=False).encode("utf-8")
        res = requests.post(url, data=data, headers=self.headers)
        if res.status_code == 200:
            data = res.json()['results']
            num = len(data)
            df = pd.DataFrame()
            for i in range(num):
                sub = pd.DataFrame(data[i]['data'])
                sub['title'] = data[i]['title']
                sub = sub[['title', 'period', 'ratio']]
                df = pd.concat([df, sub], axis=0, ignore_index=True)
            pivot = pd.pivot(df, index='period', columns='title',
                             values='ratio').reset_index()
            pivot.columns.name = None
            pivot = pivot.rename(columns={"period": "날짜"})
            return pivot

        else:
            return res

    def datalab_shopping_category_device(self, **kwargs):
        """
        쇼핑인사이트 분야 내 기기별 트렌드 조회
        - API 레퍼런스: https://developers.naver.com/docs/serviceapi/datalab/shopping/shopping.md
        """
        url = "https://openapi.naver.com/v1/datalab/shopping/category/device"
        data = {}
        for k, v in kwargs.items():
            data[k] = v
        data = json.dumps(data, ensure_ascii=False).encode("utf-8")
        res = requests.post(url, data=data, headers=self.headers)
        if res.status_code == 200:
            data = res.json()['results']
            df = pd.DataFrame(data[0]['data'])
            df = df[['group', 'period', 'ratio']]
            pivot = pd.pivot(df, index='period', columns=[
                             'group'], values='ratio').reset_index()
            pivot.columns.name = None
            pivot = pivot.rename(columns={"period": "날짜"})
            return pivot

        else:
            return res

    def datalab_shopping_category_gender(self, **kwargs):
        """
        쇼핑인사이트 분야 내 성별 트렌드 조회
        - API 레퍼런스: https://developers.naver.com/docs/serviceapi/datalab/shopping/shopping.md
        """
        url = "https://openapi.naver.com/v1/datalab/shopping/category/gender"
        data = {}
        for k, v in kwargs.items():
            data[k] = v
        data = json.dumps(data, ensure_ascii=False).encode("utf-8")
        res = requests.post(url, data=data, headers=self.headers)
        if res.status_code == 200:
            data = res.json()['results']
            df = pd.DataFrame(data[0]['data'])
            df = df[['group', 'period', 'ratio']]
            pivot = pd.pivot(df, index='period', columns=[
                             'group'], values='ratio').reset_index()
            pivot.columns.name = None
            pivot = pivot.rename(columns={"period": "날짜"})
            return pivot

        else:
            return res

    def datalab_shopping_category_age(self, **kwargs):
        """
        쇼핑인사이트 분야 내 연령별 트렌드 조회
        - API 레퍼런스: https://developers.naver.com/docs/serviceapi/datalab/shopping/shopping.md
        """
        url = "https://openapi.naver.com/v1/datalab/shopping/category/age"
        data = {}
        for k, v in kwargs.items():
            data[k] = v
        data = json.dumps(data, ensure_ascii=False).encode("utf-8")
        res = requests.post(url, data=data, headers=self.headers)
        if res.status_code == 200:
            data = res.json()['results']
            df = pd.DataFrame(data[0]['data'])
            df = df[['group', 'period', 'ratio']]
            pivot = pd.pivot(df, index='period', columns=[
                             'group'], values='ratio').reset_index()
            pivot.columns.name = None
            pivot = pivot.rename(columns={"period": "날짜"})
            return pivot

        else:
            return res

    def datalab_shopping_category_keywords(self, **kwargs):
        """
        쇼핑인사이트 키워드별 트렌드 조회
        - API 레퍼런스: https://developers.naver.com/docs/serviceapi/datalab/shopping/shopping.md
        """
        url = "https://openapi.naver.com/v1/datalab/shopping/category/keywords"
        data = {}
        for k, v in kwargs.items():
            data[k] = v
        data = json.dumps(data, ensure_ascii=False).encode("utf-8")
        res = requests.post(url, data=data, headers=self.headers)
        if res.status_code == 200:
            data = res.json()['results']
            num = len(data)
            df = pd.DataFrame()
            for i in range(num):
                sub = pd.DataFrame(data[i]['data'])
                sub['title'] = data[i]['title']
                sub = sub[['title', 'period', 'ratio']]
                df = pd.concat([df, sub], axis=0, ignore_index=True)
            pivot = pd.pivot(df, index='period', columns='title',
                             values='ratio').reset_index()
            pivot.columns.name = None
            pivot = pivot.rename(columns={"period": "날짜"})
            return pivot

        else:
            return res

    def datalab_shopping_category_keyword_device(self, **kwargs):
        """
        쇼핑인사이트 키워드 기기별 트렌드 조회
        - API 레퍼런스: https://developers.naver.com/docs/serviceapi/datalab/shopping/shopping.md
        """
        url = "https://openapi.naver.com/v1/datalab/shopping/category/keyword/device"
        data = {}
        for k, v in kwargs.items():
            data[k] = v
        data = json.dumps(data, ensure_ascii=False).encode("utf-8")
        res = requests.post(url, data=data, headers=self.headers)

        if res.status_code == 200:
            data = res.json()['results']
            df = pd.DataFrame(data[0]['data'])
            df = df[['group', 'period', 'ratio']]
            pivot = pd.pivot(df, index='period', columns=[
                             'group'], values='ratio').reset_index()
            pivot.columns.name = None
            pivot = pivot.rename(columns={"period": "날짜"})
            return pivot

        else:
            return res

    def datalab_shopping_category_keyword_gender(self, **kwargs):
        """
        쇼핑인사이트 키워드 성별 트렌드 조회
        - API 레퍼런스: https://developers.naver.com/docs/serviceapi/datalab/shopping/shopping.md
        """
        url = "https://openapi.naver.com/v1/datalab/shopping/category/keyword/gender"
        data = {}
        for k, v in kwargs.items():
            data[k] = v
        data = json.dumps(data, ensure_ascii=False).encode("utf-8")
        res = requests.post(url, data=data, headers=self.headers)

        if res.status_code == 200:
            data = res.json()['results']
            df = pd.DataFrame(data[0]['data'])
            df = df[['group', 'period', 'ratio']]
            pivot = pd.pivot(df, index='period', columns=[
                             'group'], values='ratio').reset_index()
            pivot.columns.name = None
            pivot = pivot.rename(columns={"period": "날짜"})
            return pivot

        else:
            return res

    def datalab_shopping_category_keyword_age(self, **kwargs):
        """
        쇼핑인사이트 키워드 연령별 트렌드 조회
        - API 레퍼런스: https://developers.naver.com/docs/serviceapi/datalab/shopping/shopping.md
        """
        url = "https://openapi.naver.com/v1/datalab/shopping/category/keyword/age"
        data = {}
        for k, v in kwargs.items():
            data[k] = v
        data = json.dumps(data, ensure_ascii=False).encode("utf-8")
        res = requests.post(url, data=data, headers=self.headers)

        if res.status_code == 200:
            data = res.json()['results']
            df = pd.DataFrame(data[0]['data'])
            df = df[['group', 'period', 'ratio']]
            pivot = pd.pivot(df, index='period', columns=[
                             'group'], values='ratio').reset_index()
            pivot.columns.name = None
            pivot = pivot.rename(columns={"period": "날짜"})
            return pivot

        else:
            return res

    def util_shorturl(self, url):
        """
        단축 URL 요청(JSON)
        - API 레퍼런스: https://developers.naver.com/docs/utils/shortenurl/
        """
        req_url = f"https://openapi.naver.com/v1/util/shorturl?url={url}"
        res = requests.get(req_url, headers=self.headers)
        if res.status_code == 200:
            return res.json()
        else:
            return res
        
    def papago_n2mt(self, **kwargs):
        """
        Papago 번역
        - API 레퍼런스: https://developers.naver.com/docs/papago/papago-nmt-api-reference.md
        """
        url = "https://openapi.naver.com/v1/papago/n2mt"
        data = {}
        for k, v in kwargs.items():
            data[k] = v
        data = json.dumps(data, ensure_ascii=False).encode("utf-8")
        res = requests.post(url, data=data, headers=self.headers)
        if res.status_code == 200:
            return res.json()['message']['result']['translatedText']
        else:
            return res
        
    def krdict_romanization(self, **kwargs):
        """
        한글 인명-로마자 변환
        - API 레퍼런스: https://developers.naver.com/docs/papago/papago-romanization-api-reference.md
        """
        url = "https://openapi.naver.com/v1/krdict/romanization?"
        for k, v in kwargs.items():
            url += f"{k}={v}&"
        res = requests.get(url, headers=self.headers)
        if res.status_code == 200:
            return pd.DataFrame(res.json()['aResult'][0]['aItems'])
        else:
            return res

    def search_blog(self, **kwargs):
        """
        검색 블로그
        - API 레퍼런스: https://developers.naver.com/docs/serviceapi/search/blog/blog.md
        """
        url = "https://openapi.naver.com/v1/search/blog.json?"
        for k, v in kwargs.items():
            url += f"{k}={v}&"
        res = requests.get(url, headers=self.headers)
        if res.status_code == 200:
            return pd.DataFrame(res.json()['items'])
        else:
            return res

    def search_news(self, **kwargs):
        """
        검색 뉴스
        - API 레퍼런스: https://developers.naver.com/docs/serviceapi/search/news/news.md
        """
        url = "https://openapi.naver.com/v1/search/news.json?"
        for k, v in kwargs.items():
            url += f"{k}={v}&"
        res = requests.get(url, headers=self.headers)
        if res.status_code == 200:
            return pd.DataFrame(res.json()['items'])
        else:
            return res
        
    def search_book(self, **kwargs):
        """
        검색 책
        - API 레퍼런스: https://developers.naver.com/docs/serviceapi/search/book/book.md#%EC%B1%85
        """
        url = "https://openapi.naver.com/v1/search/book.json?"
        for k, v in kwargs.items():
            url += f"{k}={v}&"
        res = requests.get(url, headers=self.headers)
        if res.status_code == 200:
            return pd.DataFrame(res.json()['items'])
        else:
            return res
        
    def search_encyc(self, **kwargs):
        """
        검색 백과사전
        - API 레퍼런스: https://developers.naver.com/docs/serviceapi/search/encyclopedia/encyclopedia.md
        """
        url = "https://openapi.naver.com/v1/search/encyc.json?"
        for k, v in kwargs.items():
            url += f"{k}={v}&"
        res = requests.get(url, headers=self.headers)
        if res.status_code == 200:
            return pd.DataFrame(res.json()['items'])
        else:
            return res

    def search_movie(self, **kwargs):
        """
        검색 영화
        - API 레퍼런스: https://developers.naver.com/docs/serviceapi/search/movie/movie.md
        """
        url = "https://openapi.naver.com/v1/search/movie.json?"
        for k, v in kwargs.items():
            url += f"{k}={v}&"
        res = requests.get(url, headers=self.headers)
        if res.status_code == 200:
            return pd.DataFrame(res.json()['items'])
        else:
            return res

    def search_cafearticle(self, **kwargs):
        """
        검색 카페글
        - API 레퍼런스: https://developers.naver.com/docs/serviceapi/search/cafearticle/cafearticle.md
        """
        url = "https://openapi.naver.com/v1/search/cafearticle.json?"
        for k, v in kwargs.items():
            url += f"{k}={v}&"
        res = requests.get(url, headers=self.headers)
        if res.status_code == 200:
            return pd.DataFrame(res.json()['items'])
        else:
            return res

    def search_kin(self, **kwargs):
        """
        검색 지식인
        - API 레퍼런스: https://developers.naver.com/docs/serviceapi/search/kin/kin.md
        """
        url = "https://openapi.naver.com/v1/search/kin.json?"
        for k, v in kwargs.items():
            url += f"{k}={v}&"
        res = requests.get(url, headers=self.headers)
        if res.status_code == 200:
            return pd.DataFrame(res.json()['items'])
        else:
            return res

    def search_webkr(self, **kwargs):
        """
        검색 웹문서
        - API 레퍼런스: https://developers.naver.com/docs/serviceapi/search/web/web.md
        """
        url = "https://openapi.naver.com/v1/search/webkr.json?"
        for k, v in kwargs.items():
            url += f"{k}={v}&"
        res = requests.get(url, headers=self.headers)
        if res.status_code == 200:
            return pd.DataFrame(res.json()['items'])
        else:
            return res

    def search_image(self, **kwargs):
        """
        검색 이미지
        - API 레퍼런스: https://developers.naver.com/docs/serviceapi/search/image/image.md
        """
        url = "https://openapi.naver.com/v1/search/image?"
        for k, v in kwargs.items():
            url += f"{k}={v}&"
        res = requests.get(url, headers=self.headers)
        if res.status_code == 200:
            return pd.DataFrame(res.json()['items'])
        else:
            return res

    def search_local(self, **kwargs):
        """
        검색 지역
        - API 레퍼런스: https://developers.naver.com/docs/serviceapi/search/local/local.md
        """
        url = "https://openapi.naver.com/v1/search/local.json?"
        for k, v in kwargs.items():
            url += f"{k}={v}&"
        res = requests.get(url, headers=self.headers)
        if res.status_code == 200:
            return pd.DataFrame(res.json()['items'])
        else:
            return res
        
    def search_shop(self, **kwargs):
        """
        검색 쇼핑
        - API 레퍼런스: https://developers.naver.com/docs/serviceapi/search/shopping/shopping.md
        """
        url = "https://openapi.naver.com/v1/search/shop.json?"
        for k, v in kwargs.items():
            url += f"{k}={v}&"
        res = requests.get(url, headers=self.headers)
        if res.status_code == 200:
            return pd.DataFrame(res.json()['items'])
        else:
            return res
        
    def search_doc(self, **kwargs):
        """
        검색 전문자료
        - API 레퍼런스: https://developers.naver.com/docs/serviceapi/search/doc/doc.md
        """
        url = "https://openapi.naver.com/v1/search/doc.json?"
        for k, v in kwargs.items():
            url += f"{k}={v}&"
        res = requests.get(url, headers=self.headers)
        if res.status_code == 200:
            return pd.DataFrame(res.json()['items'])
        else:
            return res

### 통합 검색어 트렌드 API

In [ ]:
# 네이버 API 인스턴스
naver = Naver(naver_id, naver_pw)

# 파라미터
startDate = "2022-01-01"
endDate = "2022-08-30"
timeUnit = "date"
keywordGroups = [
    {
        "groupName": "미국",
        "keywords": ["연준", "FRB", "FED", "FOMC"]
    },

    {
        "groupName": "한국",
        "keywords": ["한국은행", "금융통화위원회"]
    }
]


# 요청
df = naver.datalab_search(startDate=startDate,
                          endDate=endDate,
                          timeUnit=timeUnit,
                          keywordGroups=keywordGroups)
df

### Shopping Insight API 1

In [ ]:
# 네이버 API 인스턴스
naver = Naver(naver_id, naver_pw)

# 파라미터
startDate = "2022-01-01"
endDate = "2022-08-30"
timeUnit = "date"
category = [
    {
        "name": "카테고리1",
        "param": ["50000000"],
    },
    {
        "name": "카테고리2",
        "param": ["50000002"],
    },
]

# 요청
df = naver.datalab_shopping_categories(startDate=startDate,
                                       endDate=endDate,
                                       timeUnit=timeUnit,
                                       category=category)
df

### Shopping Insight API 2

In [ ]:
# 네이버 API 인스턴스
naver = Naver(naver_id, naver_pw)

# 파라미터
startDate = "2022-01-01"
endDate = "2022-08-30"
timeUnit = "date"
category = "50000002"

# 요청
df = naver.datalab_shopping_category_device(startDate=startDate,
                                            endDate=endDate,
                                            timeUnit=timeUnit,
                                            category=category)
df

### Shopping Insight API 3

In [ ]:
# 네이버 API 인스턴스
naver = Naver(naver_id, naver_pw)

# 파라미터
startDate = "2022-01-01"
endDate = "2022-08-30"
timeUnit = "date"
category = "50000002"

# 요청
df = naver.datalab_shopping_category_gender(startDate=startDate,
                                            endDate=endDate,
                                            timeUnit=timeUnit,
                                            category=category)
df

### Shopping Insight API 4

In [ ]:
# 네이버 API 인스턴스
naver = Naver(naver_id, naver_pw)

# 파라미터
startDate = "2022-01-01"
endDate = "2022-08-30"
timeUnit = "date"
category = "50000002"

# 요청
df = naver.datalab_shopping_category_age(startDate=startDate,
                                         endDate=endDate,
                                         timeUnit=timeUnit,
                                         category=category)
df

### Shopping Insight API 5

In [ ]:
# 네이버 API 인스턴스
naver = Naver(naver_id, naver_pw)

# 파라미터
startDate = "2022-01-01"
endDate = "2022-08-30"
timeUnit = "date"
category = "50000000"
keyword = [
    {
        "name": "키워드1",
        "param": ["정장"],
    },
    {
        "name": "키워드2",
        "param": ["비즈니스 캐주얼"],
    },
]

# 요청
df = naver.datalab_shopping_category_keywords(startDate=startDate,
                                              endDate=endDate,
                                              timeUnit=timeUnit,
                                              category=category,
                                              keyword=keyword)
df

### Shopping Insight API 6

In [ ]:
# 네이버 API 인스턴스
naver = Naver(naver_id, naver_pw)

# 파라미터
startDate = "2022-01-01"
endDate = "2022-08-30"
timeUnit = "date"
category = "50000000"
keyword = "정장"

# 요청
df = naver.datalab_shopping_category_keyword_device(startDate=startDate,
                                                    endDate=endDate,
                                                    timeUnit=timeUnit,
                                                    category=category,
                                                    keyword=keyword)
df

### Shopping Insight API 7

In [ ]:
# 네이버 API 인스턴스
naver = Naver(naver_id, naver_pw)

# 파라미터
startDate = "2022-01-01"
endDate = "2022-08-30"
timeUnit = "date"
category = "50000000"
keyword = "정장"

# 요청
df = naver.datalab_shopping_category_keyword_gender(startDate=startDate,
                                                    endDate=endDate,
                                                    timeUnit=timeUnit,
                                                    category=category,
                                                    keyword=keyword)
df

### Shopping Insight API 8

In [ ]:
# 네이버 API 인스턴스
naver = Naver(naver_id, naver_pw)

# 파라미터
startDate = "2022-01-01"
endDate = "2022-08-30"
timeUnit = "date"
category = "50000000"
keyword = "정장"

# 요청
df = naver.datalab_shopping_category_keyword_age(startDate=startDate,
                                                 endDate=endDate,
                                                 timeUnit=timeUnit,
                                                 category=category,
                                                 keyword=keyword)
df

### 단축 URL API

In [ ]:
# 네이버 API 인스턴스
naver = Naver(naver_id, naver_pw)

# 파라미터
url = "https://wooiljeong.github.io"

# 단축 URL API 호출
res = naver.util_shorturl(url)
res

### Papago 번역 API

In [ ]:
# 네이버 API 인스턴스
naver = Naver(naver_id, naver_pw)

# 파라미터
source = "ko"
target = "en"
text = "파이썬으로 파파고 번역 API를 호출해봅니다."

# 단축 URL API 호출
res = naver.papago_n2mt(source = source, 
                        target = target, 
                        text = text)
res

### 한글 인명 로마자 변환 API

In [ ]:
# 네이버 API 인스턴스
naver = Naver(naver_id, naver_pw)

# 파라미터
query = "홍길동"

# 단축 URL API 호출
df = naver.krdict_romanization(query=query)
df

### 검색 블로그

In [ ]:
# 네이버 API 인스턴스
naver = Naver(naver_id, naver_pw)

# 파라미터
query = "아파트"
display = 100
sort = "sim" # date

# 단축 URL API 호출
df = naver.search_blog(query=query, display=display, sort=sort)
df

### 검색 뉴스

In [ ]:
# 네이버 API 인스턴스
naver = Naver(naver_id, naver_pw)

# 파라미터
query = "아파트"
display = 100
sort = "sim" # date

# 단축 URL API 호출
df = naver.search_news(query=query, display=display, sort=sort)
df

### 검색 책

In [ ]:
# 네이버 API 인스턴스
naver = Naver(naver_id, naver_pw)

# 파라미터
query = "사피엔스"
display = 100
sort = "sim" # date

# 단축 URL API 호출
df = naver.search_book(query=query, display=display, sort=sort)
df

### 검색 백과사전

In [ ]:
# 네이버 API 인스턴스
naver = Naver(naver_id, naver_pw)

# 파라미터
query = "사피엔스"
display = 100
sort = "sim" # date

# 단축 URL API 호출
df = naver.search_encyc(query=query, display=display, sort=sort)
df

### 검색 영화

In [ ]:
# 네이버 API 인스턴스
naver = Naver(naver_id, naver_pw)

# 파라미터
query = "인터스텔라"
display = 100
sort = "sim" # date

# 단축 URL API 호출
df = naver.search_movie(query=query, display=display, sort=sort)
df

### 검색 카페글

In [ ]:
# 네이버 API 인스턴스
naver = Naver(naver_id, naver_pw)

# 파라미터
query = "퀀트"
display = 100
sort = "sim" # date

# 단축 URL API 호출
df = naver.search_cafearticle(query=query, display=display, sort=sort)
df

### 검색 지식인

In [ ]:
# 네이버 API 인스턴스
naver = Naver(naver_id, naver_pw)

# 파라미터
query = "퀀트"
display = 100
sort = "sim" # date

# 단축 URL API 호출
df = naver.search_kin(query=query, display=display, sort=sort)
df

### 검색 지역

In [ ]:
# 네이버 API 인스턴스
naver = Naver(naver_id, naver_pw)

# 파라미터
query = "스타벅스"
display = 5
sort = "comment" # random(유사도순), comment(카페/블로그 리뷰 개수 순)

# 단축 URL API 호출
df = naver.search_local(query=query, display=display, sort=sort)
df

### 검색 웹문서

In [ ]:
# 네이버 API 인스턴스
naver = Naver(naver_id, naver_pw)

# 파라미터
query = "힙합"
display = 100
sort = "sim" # sim, date, asc, dsc

# 단축 URL API 호출
df = naver.search_webkr(query=query, display=display, sort=sort)
df

### 검색 이미지

In [ ]:
# 네이버 API 인스턴스
naver = Naver(naver_id, naver_pw)

# 파라미터
query = "고양이"
display = 100
sort = "sim"

# 단축 URL API 호출
df = naver.search_image(query=query, display=display, sort=sort)
df

### 검색 쇼핑

In [ ]:
# 네이버 API 인스턴스
naver = Naver(naver_id, naver_pw)

# 파라미터
query = "아이패드"
exclude = "used:rental:cbshop"
display = 100
sort = "sim" # sim, date, asc, dsc

# 단축 URL API 호출
df = naver.search_shop(query=query, exclude=exclude, display=display, sort=sort)
df

### 검색 전문자료

In [ ]:
# 네이버 API 인스턴스
naver = Naver(naver_id, naver_pw)

# 파라미터
query = "부동산"
display = 100
sort = "sim"

# 단축 URL API 호출
df = naver.search_doc(query=query, display=display, sort=sort)
df

## NAVER CLOUD PLATFORM

In [ ]:
class NaverCloudPlatform:
    """
    네이버 클라우드 플랫폼 OPEN API 클래스
    """
    def __init__(self, client_id, client_secret):
        """
        API 인증 정보 초기화
        """
        self.headers = {
            "X-NCP-APIGW-API-KEY-ID": client_id,
            "X-NCP-APIGW-API-KEY": client_secret,
        }
        
    def geocoding(self, query):
        """
        지오코딩 API
        - API 레퍼런스: https://api.ncloud-docs.com/docs/ai-naver-mapsgeocoding
        """
        url = f"https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode?query={query}"
        res = requests.get(url, headers=self.headers)
        if res.status_code == 200:
            return res.json()
        else:
            return res

    def reverse_geocoding(self, lon, lat):
        """
        리버스 지오코딩 API
        - API 레퍼런스: https://api.ncloud-docs.com/docs/ai-naver-mapsreversegeocoding
        """
        coords = f"{lon},{lat}"
        url = f"https://naveropenapi.apigw.ntruss.com/map-reversegeocode/v2/gc?coords={coords}&output=json&orders=addr"
        res = requests.get(url, headers=self.headers)
        if res.status_code == 200:
            return res.json()
        else:
            return res
        
    def directions5(self, **kwargs):
        """
        Directions5 API
        - API 레퍼런스: https://api.ncloud-docs.com/docs/ai-naver-mapsdirections
        """
        url = f"https://naveropenapi.apigw.ntruss.com/map-direction/v1/driving?"
        for k, v in kwargs.items():
            url += f"{k}={v}&"
        res = requests.get(url, headers=self.headers)
        if res.status_code == 200:
            return res.json()
        else:
            return res

    def directions15(self, **kwargs):
        """
        Directions15 API
        - API 레퍼런스: https://api.ncloud-docs.com/docs/ai-naver-mapsdirections15
        """
        url = f"https://naveropenapi.apigw.ntruss.com/map-direction-15/v1/driving?"
        for k, v in kwargs.items():
            url += f"{k}={v}&"
        res = requests.get(url, headers=self.headers)
        if res.status_code == 200:
            return res.json()
        else:
            return res

### Directions5

In [ ]:
# 클라우드 API 인스턴스
ncp = NaverCloudPlatform(cloud_id, cloud_pw)

# 파라미터
start = "127.1115061,37.3864539"
goal = "127.01953476,37.5160802"

# 지오코딩 요청
res = ncp.directions5(start=start, goal=goal)
res

### Directions15

In [ ]:
# 클라우드 API 인스턴스
ncp = NaverCloudPlatform(cloud_id, cloud_pw)

# 파라미터
start = "127.1115061,37.3864539"
goal = "127.01953476,37.5160802"

# 지오코딩 요청
res = ncp.directions15(start=start, goal=goal)
res

### Geocoding

In [ ]:
# 클라우드 API 인스턴스
ncp = NaverCloudPlatform(cloud_id, cloud_pw)

# 주소
query = "해밀동 산8"

# 지오코딩 요청
res = ncp.geocoding(query)
res

### Reverse Geocoding

In [ ]:
# 클라우드 API 인스턴스
ncp = NaverCloudPlatform(cloud_id, cloud_pw)

# 경도, 위도
lon, lat = 127.228954, 37.376474

# 지오코딩 요청
res = ncp.reverse_geocoding(lon, lat)
res